In [1]:
import numpy as np
import pandas as pd
from keras.layers import LSTM, Bidirectional, Dense, TimeDistributed
from keras.models import Sequential

Using TensorFlow backend.


In [ ]:
# Specification of directionality in keras

model = Sequential()
model.add(LSTM(...,..., go_backwards = True))

# Bidirectional first layer wraps first LSTM layer for both 
# directions. This layer merges the output from two parallel LSTMs,
# one with input processed forward and one with output 
# processed backwards.

model = Sequential()
model.add(Bidirectional(LSTM(...), input_shape=(...)))

# Cumulative Sum Prediction Problem
The problem is defined as a sequence of random values between 0 and 1. This sequence is taken
as input for the problem with each number provided once per time step. A binary label (0 or 1)
is associated with each input. The output values are all 0. Once the cumulative sum of the
input values in the sequence exceeds a threshold, then the output value 
flips from 0 to 1.
A threshold of one quarter the sequence length is used.

In [2]:
X = np.random.rand(10)

In [3]:
limit = 10/4

In [4]:
y = np.array([0 if x < limit else 1 for x in np.cumsum(X)])

In [5]:
print("Example input values: ", X ,
      '\n',
      "Example output values: ", y)

Example input values:  [ 0.68968101  0.6902787   0.71390136  0.5410991   0.19614277  0.58387646
  0.3721204   0.87764648  0.53272351  0.87801889] 
 Example output values:  [0 0 0 1 1 1 1 1 1 1]


In [6]:
def generate_seq(n_sequences, n_timesteps):
    '''
    Function for generating sequences we use as data.
    '''
    seqX, seqY = [], []
    
    for i in range(n_sequences):
        X = np.random.rand(n_timesteps)
        limit = n_timesteps/4
        y = np.array([0 if x < limit else 1 for x in np.cumsum(X)])
        
        seqX.append(X)
        seqY.append(y)
        
    seqX = np.array(seqX).reshape(n_sequences, n_timesteps, 1)
    seqY = np.array(seqY).reshape(n_sequences, n_timesteps, 1)

    return seqX, seqY

In [7]:
# Define input timesteps (sequence length)
n_timesteps = 10

In [8]:
# Define model and print summary
model = Sequential()
model.add(Bidirectional(LSTM(50, return_sequences=True), input_shape=(n_timesteps, 1)))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 10, 100)           20800     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 1)             101       
Total params: 20,901
Trainable params: 20,901
Non-trainable params: 0
_________________________________________________________________


In [9]:
# We can simplify training by using the number of randomly generated
# sequences as a proxy for epochs. This allows us to generate a large 
# number of examples, in this case 50,000, store them in memory,
# and fit them in one Keras epoch.
X, y = generate_seq(50000, n_timesteps)
model.fit(X, y, epochs=1, batch_size=10)

Epoch 1/1
50000/50000 [==============================] - 184s 4ms/step - loss: 0.0495 - acc: 0.9822


In [16]:
# Evaluate model on 1000 sequences
X, y = generate_seq(1000, n_timesteps)
loss, acc = model.evaluate(X,y)

print('Loss: %f, Accuracy: %f' % (loss, acc*100))

1000/1000 [==============================] - 1s 722us/step
Loss: 0.018647, Accuracy: 99.630000


In [14]:
# Predict values for 10 sequences and print information
for _ in range(10):
    X, y = generate_seq(1, n_timesteps)
    yhat = model.predict_classes(X, verbose=0)
    exp, pred = y.reshape(n_timesteps), yhat.reshape(n_timesteps)
    print('y=%s, yhat=%s, correct=%s' % (exp, pred, np.array_equal(exp,pred)))

y=[0 0 0 0 1 1 1 1 1 1], yhat=[0 0 0 0 1 1 1 1 1 1], correct=True
y=[0 0 0 0 1 1 1 1 1 1], yhat=[0 0 0 0 1 1 1 1 1 1], correct=True
y=[0 0 0 0 1 1 1 1 1 1], yhat=[0 0 0 0 1 1 1 1 1 1], correct=True
y=[0 0 0 0 1 1 1 1 1 1], yhat=[0 0 0 0 1 1 1 1 1 1], correct=True
y=[0 0 0 0 0 1 1 1 1 1], yhat=[0 0 0 0 0 1 1 1 1 1], correct=True
y=[0 0 0 1 1 1 1 1 1 1], yhat=[0 0 0 1 1 1 1 1 1 1], correct=True
y=[0 0 0 1 1 1 1 1 1 1], yhat=[0 0 0 1 1 1 1 1 1 1], correct=True
y=[0 0 0 0 0 0 1 1 1 1], yhat=[0 0 0 0 0 0 1 1 1 1], correct=True
y=[0 0 1 1 1 1 1 1 1 1], yhat=[0 0 1 1 1 1 1 1 1 1], correct=True
y=[0 0 0 1 1 1 1 1 1 1], yhat=[0 0 0 1 1 1 1 1 1 1], correct=True
